<a href="https://colab.research.google.com/github/sebrojas/KerasDay/blob/main/KERAS_Day.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -q https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins.csv -O /tmp/penguins.csv

In [ ]:
# Install TensorFlow Dececision Forests
!pip install tensorflow_decision_forests

In [ ]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

In [ ]:
dataset_df = pd.read_csv("/tmp/penguins.csv")

# Display the first 3 examples.
dataset_df.head(3)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007


In [ ]:
label = "species"

for col in dataset_df.columns:
  if dataset_df[col].dtype not in [str, object]:
    dataset_df[col] = dataset_df[col].fillna(0)

def split_dataset(dataset, test_ratio=0.30):
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, test_ds_pd = split_dataset(dataset_df)

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label)

In [ ]:
input_1 = tf.keras.Input(shape=(1,), name="bill_length_mm", dtype="float")
input_2 = tf.keras.Input(shape=(1,), name="island", dtype="string")

nn_raw_inputs = [input_1, input_2]

Normalization = tf.keras.layers.Normalization
CategoryEncoding = tf.keras.layers.CategoryEncoding
StringLookup = tf.keras.layers.StringLookup

values = train_ds_pd["bill_length_mm"].values[:, tf.newaxis]
input_1_normalizer = Normalization()
input_1_normalizer.adapt(values)

values = train_ds_pd["island"].values
input_2_indexer = StringLookup(max_tokens=32)
input_2_indexer.adapt(values)

input_2_onehot = CategoryEncoding(output_mode="binary", max_tokens=32)

normalized_input_1 = input_1_normalizer(input_1)
normalized_input_2 = input_2_onehot(input_2_indexer(input_2))

nn_processed_inputs = [normalized_input_1, normalized_input_2]

y = tf.keras.layers.Concatenate()(nn_processed_inputs)
y = tf.keras.layers.Dense(16, activation=tf.nn.relu6)(y)
last_layer = tf.keras.layers.Dense(8, activation=tf.nn.relu, name="last")(y)

classification_output = tf.keras.layers.Dense(3)(y)

nn_model = tf.keras.models.Model(nn_raw_inputs, classification_output)

In [ ]:
nn_without_head = tf.keras.models.Model(inputs=nn_model.inputs, outputs=last_layer)
df_and_nn_model = tfdf.keras.RandomForestModel(preprocessing=nn_without_head)

Use /tmp/tmpnebjicjl as temporary training directory


In [ ]:
nn_model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=["accuracy"])

nn_model.fit(x=train_ds, validation_data=test_ds, epochs=10)
nn_model.summary()

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/functional.py:642: UserWarning: Input dict contained keys ['bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex', 'year'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


1/1 [==============================] - 1s 911ms/step - loss: 1.1218 - accuracy: 0.2563 - val_loss: 1.1077 - val_accuracy: 0.2547
Epoch 2/10
1/1 [==============================] - 0s 24ms/step - loss: 1.1165 - accuracy: 0.2605 - val_loss: 1.1029 - val_accuracy: 0.2642
Epoch 3/10
1/1 [==============================] - 0s 24ms/step - loss: 1.1113 - accuracy: 0.2605 - val_loss: 1.0981 - val_accuracy: 0.2830
Epoch 4/10
1/1 [==============================] - 0s 24ms/step - loss: 1.1061 - accuracy: 0.2647 - val_loss: 1.0934 - val_accuracy: 0.3019
Epoch 5/10
1/1 [==============================] - 0s 24ms/step - loss: 1.1010 - accuracy: 0.2689 - val_loss: 1.0887 - val_accuracy: 0.3019
Epoch 6/10
1/1 [==============================] - 0s 25ms/step - loss: 1.0959 - accuracy: 0.2689 - val_loss: 1.0840 - val_accuracy: 0.3019
Epoch 7/10
1/1 [==============================] - 0s 24ms/step - loss: 1.0908 - accuracy: 0.2731 - val_loss: 1.0793 - val_accuracy: 0.3019
Epoch 8/10
1/1 [=====================

In [ ]:
df_and_nn_model.fit(x=train_ds)

Reading training dataset...
Training dataset read in 0:00:06.735766. Found 238 examples.
Training model...
Model trained in 0:00:00.093470
Compiling model...
Model compiled.


In [ ]:
df_and_nn_model.compile(metrics=["accuracy"])
print("NeuralNetwork + DecisionForest:", df_and_nn_model.evaluate(test_ds))
print("NeuralNetwork Sola :", nn_model.evaluate(test_ds))

1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 0.9245
NeuralNetwork + DecisionForest: [0.0, 0.9245283007621765]
1/1 [==============================] - 0s 52ms/step - loss: 1.0656 - accuracy: 0.3019
NeuralNetwork Sola : [1.065618872642517, 0.30188679695129395]
